In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm
import time
import pickle

### Get each link from the main page of the archive.

In [ ]:
url = "https://2e.aonprd.com/Rules.aspx"
response = requests.get(url)
html_text = response.text
rules_numbers = [1, 2, 6, 12, 17, 22, 39, 48, 74, 75, 85, 88, 94, 96, 98, 100, 113, 118, 123, 129, 150, 151, 146, 191]
span_id = 'ctl00_RadDrawer1_Content_MainContent_DetailedOutput'

In [ ]:
def generate_rules_list(html_text, slice_number):
    slice_rules_link_list = []
    
    soup = BeautifulSoup(html_text, 'html.parser')
    div = soup.find('div', id=f'SO_Rules{slice_number}')
    div = str(div)
    
    soup = BeautifulSoup(div, 'html.parser')
    anchors = soup.find_all('a')
    
    for anchor in anchors:
        href = anchor.get('href')
        text = anchor.text
        slice_rules_link_list.append({text: {'link': href}})
    
    return slice_rules_link_list

In [ ]:
all_rules_link_list = []
for rule_number in rules_numbers:
    slice_rules_link_list = generate_rules_list(html_text, rule_number)
    all_rules_link_list += slice_rules_link_list

In [ ]:
all_rules_link_list

### Get information from the list up to the next title.

In [ ]:
def href_clean(href):
    if href.name == 'u':
        return href.find('a').get('href')
    elif href.name == 'a':
        return href.get('href')
    
def remove_piazo_links(hrefs):
    for href_dict in hrefs:
        for href in href_dict:
            if href == 'paizo.com':
                del [hrefs[hrefs.index(href_dict)]]
    return hrefs

In [ ]:
def get_rule_information(url):
    try:
        response = requests.get(url)
        html_text = response.text

        soup = BeautifulSoup(html_text, 'html.parser')
        span = soup.find('span', id=span_id)
        soup = BeautifulSoup(str(span), 'html.parser')

        # Find the first <h1> tag in the HTML
        start_tag = span.contents[0]

        # Find the second <h1> tag in the HTML
        h1_tags = soup.find_all('h1')
        if len(h1_tags) < 2:
            second_h1_tag = None
        else:
            second_h1_tag = h1_tags[1]

        hrefs = [start_tag]
        rule_text = ""
        sibling = start_tag.next_sibling
        while sibling is not None and sibling != second_h1_tag:
            if sibling.name is None:
                rule_text += sibling.strip() if sibling.strip() != '' else ''
            elif sibling.name == 'i':
                rule_text += ' ' + sibling.text.strip()
            elif sibling.name == 'a':
                hrefs.append(sibling)
            elif sibling.name == 'u':
                hrefs.append(sibling)
            sibling = sibling.next_sibling

        return {'title': h1_tags[0].text,
                'book referance': hrefs[-1].text,
                'rule text': re.sub(r'//', '', rule_text),
                'tree_links': remove_piazo_links([{ref.text: href_clean(ref)} for ref in hrefs[:-1]])
               }
    except Exception as e:
        return {'Error': 'Error'}

In [ ]:
thing = get_rule_information('https://2e.aonprd.com/Rules.aspx?ID=341')

In [ ]:
thing

In [ ]:
html_text

In [ ]:
response = requests.get(f'https://2e.aonprd.com/Rules.aspx?ID={len(rules_list) + 3}')
print(f'https://2e.aonprd.com/Rules.aspx?ID={len(rules_list) + 2}')
html_text = response.text

soup = BeautifulSoup(html_text, 'html.parser')
span = soup.find('span', id=span_id)
soup = BeautifulSoup(str(span), 'html.parser')

# Find the first <h1> tag in the HTML
start_tag = span.contents[0]

# Find the second <h1> tag in the HTML
h1_tags = soup.find_all('h1')
if len(h1_tags) < 2:
    second_h1_tag = None
else:
    second_h1_tag = h1_tags[1]

hrefs = [start_tag]
rule_text = ""
sibling = start_tag.next_sibling
while sibling is not None and sibling != second_h1_tag:
    if sibling.name is None:
        rule_text += sibling.strip() if sibling.strip() != '' else ''
    elif sibling.name == 'i':
        rule_text += ' ' + sibling.text.strip()
    elif sibling.name == 'a':
        hrefs.append(sibling)
    elif sibling.name == 'u':
        hrefs.append(sibling)
    sibling = sibling.next_sibling

{'title': h1_tags[0].text,
        'book referance': hrefs[-1].text,
        'rule text': re.sub(r'//', '', rule_text),
        'tree_links': remove_piazo_links([{ref.text: href_clean(ref)} for ref in hrefs[:-1]])
       }

### Download all the rules

In [ ]:
max_rule = 1975
base_url = 'https://2e.aonprd.com/'
rule_url = 'Rules.aspx?ID='

In [ ]:
rules_list = []
with open('rules_list.pickle', 'wb') as f:
    pickle.dump(rules_list, f)

In [ ]:
with open('rules_list.pickle', 'rb') as f:
    rules_list = pickle.load(f)

for i in tqdm(range(len(rules_list) + 1, max_rule + 1)):
    rules_list.append(get_rule_information(base_url + rule_url + str(i)))
    time.sleep(1)

In [ ]:
with open('rules_list.pickle', 'wb') as f:
    pickle.dump(rules_list, f)